# Linear Classification

## Interpreting a linear classifier

Notice that all linear classifiers compute the score of a class as a weighted sum of all of its pixel values across all 3 of its color channels. Depending on precisely what values we set for these weights, the function has the capacity to like or dislike certain colors at certain positions in the image. For instance you can image the "ship" class might be more likely if there is a lot of blue on the sides of an image (corresponding to water). You might want expect that the 'ship' classifier would then have a lot of positive weights across it's blue channel weights, and negative weights in the red/gree channels.

![](http://cs231n.github.io/assets/imagemap.jpg)

__Analogy of images as high-dimensional points.__ Since the images are stretched into a high dimensional column vectors, we can interpret each image as a single point in this space. Since we defined the score of each class as a weighted sum of all image pixels, each class score is a linear function over this space. We cannot visualize 3072-dimensional spaces, but if we we imagine squashing all those dimensions into onluy two dimensions. then we can try to visualize what the classifier might be doing:

![](http://cs231n.github.io/assets/pixelspace.jpeg)

**Interpretation of Linear classifiers as template matching**: Another interpretation of the weights `w` is that each row of `W` corresponds to a template of for one of the classes. The score of each class for an image is then obtained by comparing each template with the image using an _inner product_ one by one to find the one that "fits" best. With this terminology, the linear classifier is doing template matching, where the templates are learned. Another way to think of that is that we are still effectively doing Nearest Neigbor, but instead of having thousands of training images we are only using a single image per class (although we will learn it, and it does not necessarily have to be one of the images in the training set,) and we can use the inner product as the distance instead of L1 or L2 distance

Example Below: 
![](http://cs231n.github.io/assets/templates.jpg)

* Skipping ahead a bit: Example learned weights at the end of learning for CIFAR-10. Note that, for example,t he ship template contains a lot of blue pixels as expected. This template will therefore give a high score once it is matched against images of ships on the ocean with an inner product

***

Additionally, note that the horse template seems to contain a two-headed horse, which is due to both left and right facing horses in the dataset. The linear classifier **merges** these two modes of horses in the data into a single template. Similarly, the car classifier seems to have merged several modes into a single template which has to identify cars from alls ides, and of all colors. In particular, this template ended up being red, which hints that there are more red cards in the CIFAR-10 dataset than of any other color. The linear classifier is too weak to properly account for different-colored cards, but as we will see later neural networks will allow us to perform this task. Looking ahead a bit, a neural network will be able to develop intermediate neurons in its hidden layers that could detect specific car types (e.g. green car facing left, blue car facing fron, etc) and neurons on the next layer could combine these into a more accurate car score through a weighted sum of the individual car detectors.

**Bias trick.** Before moving on we want to mention a common simplifying trick to representing the two parameters **W, b** as one. Recall that we defined the score function as:

$$ f(x_i, W, b) =  W x_i + b$$

As we process through the material it is a little cumbersome to keep track of two sets of parameters (the bias **b** and weights **W**) separately. A commonly used trick is to combine the two sets of parameters into a single matrix that holds both of them by extending the vector $x_i$ with one additional dimension that always holds the constant `1` - a default _bias dimension_. With the extra dimension, the new score function will simplify to a single matrix multiplication:

$$f(x_i, W) =  W x_i$$

To illustrate this,
With our CIFAR-10 example, $x_1$ is now [3073 x 1] insteaf of [3072 x 1] - (with the extra dimension holding the constant, 1), and `W` is now [10 x 3073] instead of [10 x 3072]. The extra column that `W` now corresponds to the bias `b`. An illustration might help clarify:

![](http://cs231n.github.io/assets/wb.jpeg)

* Illustration of the bias trick. Doing a matrix multiplication and then adding a bias vector (left) is equivalent to adding a bias dimension with a constant of 1 to all input vectors and extending the weight matrix by 1 column - a bias column (right). Thus, if we preprocess our data by appending ones to all vectors we only have to learn a single matrix of weights instead of two matrices that hold the weights and the biases.

**Image data processing** As a quick note, in the example above we used the raw pixel values (which range from ([0... 255])). In Machine Learning, it is very common practice to always perform normalization on your input features (in the case of images, every pixel is through of as a feature). In particular, it is important to **center your data** by subtracting the mean from every feature. In the case of images, this corresponds to computing a *mean image* across the training images and subtracting it from every image to get images where the pixels range from approximately [-127... 127]. Further common preprocessing is to scale each input feature so that its values range from [-1,1]. Of these, zero mean centering is arguably more important but we will have to wait for its justification until we understand the dunamics of gradient descent.

## Loss function

In the previous section we defined a function from the pixel values to a class score, which was parameterized by a set of weights `W`. Moreover, we saw that we don't have control over the data **$(x_i,y_i)$** (it is fixed and given), but we do have control over these weights and we want to set the so that the prediction class scores are consistent with the ground truth labels in the training data.

For example, going back to the example images of a cat and its scores for the classes "cat", "dog" and "ship", we saw that the particular set of weights in that example was not very good at all: We fed in the pixels that depicts a cat but the cat score came out very low (-96.8) compared to the other classes (dog score 437.9 and ship score 61.95). We are going to measure out unhappiness with outcomes such as this one with a **loss function**. Intuitively, the loss will be high if we're doing a poor job of classifying the training data, and it will low if we're doing a good job.

## Multiclass Support Vector Machine Loss

There are several ways to define the details of the loss function. As a first example we will first develop a commonly used loss called the **Multiclass Support Vector Machine** (SVM) loss. The SVm loss is set up so that the SVM "wants" the correct class for each image to have a score higher than the incorrect classes by some fixed margin $\Delta$. Notice that it's sometimes helpful to antripomorphise the loss function as we did above. The SVM "wants" a certain outcome in the sense that the outcome would yield a lower loss.

Let's now get more precise. Recall that for the i-th example we are given the pixels of image $x_i$ and the label $y_i$ that specifies the index of the correct class. The score function takes the pixels and computes the vector $f(x_i, W) =  W x_i$ of the class scores, which we will abbreviate to s (short for scores). For example, the score for the j-th class is the j-th element wheren  $s_j = f(x_i, W) =  W x_i$. The multiclass SVM loss for the i-th example is then formalized as follows:

$$L_i = \sum_{j\neq y_i} \max(0, s_j - s_{y_i} + \Delta)$$

**Example** Lets unpack this with an example to see how it works. Suppose that we have three classes that recieve the score **s = [13,-7,11]** and that the first class is the true class $(i.e. y_i = 0)$. Also assume that $\Delta$ (a hyperparameter we will go into more detail soon) is 10. The expression above sums over all incorrect classes $(j \neq y_i)$, so we get two terms

$$ L_i = \max(0, -7 - 13 + 10) + \max(0, 11 - 13 + 10)$$

You can see that the first term gives zero since [- 7 - 13 + 10] gives a negative number, which is then thresholded to zero with a max(0,-) function. We get zero loss for this pair because the correct class score (13) was greater than than the incorrect by at least a margin on 10. In fact the difference was 20, which is much greater than 10 but the SVM only cares that the difference is at least 10; Any additional difference above the margin is clamped at zero with the max operation. The second term computes [11 - 13 + 10] which gives 8. That is, even though the correct class had a higher score than the incorrect class (13 > 11), it was not greater than the desired margin of 10. The difference is only 2, which is why the loss comes out to 8 (i.e. how much higher the difference would have to be to meet the margin).


* In summary, the SVM loss function wants the score of the correct class $y_i$ to be larger than the incorrect class score by at least $\Delta$ (delta).  If this is not the case, we will accumulate loss

Note that in this particular module we are working with linear score functions $(f(x_i;W) = Wx_i)$, so we can also write the loss function in this equivalent form:

$$L_i = \sum_{j\neq y_i} \max(0, w_j^T x_i - w_{y_i}^T x_i + \Delta)$$

where $w_j$ is the j-th row of `W` reshaped as a column. However, this will not necessarily be the case once we start to consider more complex forms of the score function **f**.

A Last piece of terminology, we'll mention before we finish with this section is that the threshold at zero **max(0,-)** function is often called the **hinge loss**. You'll sometimes hear about people instead using the squared hinge loss SVM (OR L2-SVM), which uses the form $max(0,-)^2$ that penalizes violated margins more strongly (quadratically instead of linearly)/ The unsquared version is more standard, but in some datasets the squared hinge loss can work better. This can be determined during cross-validation.

![](http://cs231n.github.io/assets/margin.jpg)

* The Multiclass Support Vector Machine "wants" the score of the correct class to be higher than all other scores by at least a margin of delta. If any class has a score inside the red region (or higher), then there will be accumulated loss. Otherwise the loss will be zero. Our objective will be to find the weights that will simultaneously satisfy this constraint for all examples in the training data and give a total loss that is as low as possible.

**Regularization:** There is one bug with the loss function we presented above. Suppose that we have a dataset and a set of parameters *W* that correctly classify every example (i.e. all scores are so that the margins are met, and $L_i = 0$ for all $i $). The issues is that this set of *W* is not necessarily unqiue: there might be many similar *W* that correctly classify the examples. One easy way to see this is that if some parameters *W* correctly classify all examples (so loss is zero for each example), then any multiple of these parameteres $\lambda W $ where $\lambda > 1$ will also give zero loss because this transformation uniformly stretches all scores magnitudes and hence also their abosulute differences. For example if the difference in scores between a correct class and a nearest incorrect class was 15, then multiplying all elemends of *W* by 2 would make the new difference 30. 

In other words, we wish to encode some perference for certain set of weights *W* over others to remove this ambiquity. We can do so by extending the loss function with a **regularization penalty R(W)**. The most common regularization penalty is the **L2** norm that discourages large weights through an elementwise quadratic penalty over all parameters:


$$R(W) = \sum_k\sum_l W_{k,l}^2$$

In the expression above, we are summing up all the squared elements of *W*. Notice that the regularization function is not a function of the data, it is only based on the weights. Inclhding the regularization penalty completes the Multiclass Support Vector Machine loss, which is made up of two componenets: the **data loss** (which is the average $L_i$ over all examples) and the **regularization loss**. That is, the full Multiclass SVM loss becomes:

$$L =  \underbrace{ \frac{1}{N} \sum_i L_i }_\text{data loss} + \underbrace{ \lambda R(W) }_\text{regularization loss} $$

Or expanding this out in its full form:

$$L = \frac{1}{N} \sum_i \sum_{j\neq y_i} \left[ \max(0, f(x_i; W)_j - f(x_i; W)_{y_i} + \Delta) \right] + \lambda \sum_k\sum_l W_{k,l}^2$$

Where $N$ is the number of training examples. As you can see, we append the regularization penalty to the loss objective, weighted by a hyperparameter $\lamda$. There is no simple way of setting this hyperparameter and it is usually determined by cross-validation.

In addition to the motivation we provided above there are many desirable properties to include the regularization penalty, many of which we will come back to in later sections.

## Practical Considerations

__Setting Delta.__ Note that we brushed over the hyperparameter $\Delta$ and its setting. What value should it be set to, and do we have to cross-validate it? It turns out that this hyperparameter can safely be set to $\Delta = 1$ in all cases. The hyperparameters $\Delta$ and $\lamda$ seem like two different hyperparameters, but in fact they both control the same tradeoff: The tradeoff between the data loss and the regularization loss in the objective. The key to understanding this is that the magnitude of the weights **W** has a direct effect on the scores: As we shrink all values inside **W** the score differences will become lower, and as we scale up the weights the score differences will all become higher. Therefore, the exact value of the margin between the scores $\Delta$ is in some case meaningless.

## Softmax Classifier

It turns out that the SVM is one of two commonly seen classifiers. The other popular choice is the **Softmax classifier**, which has a different loss function. If you've heard of the binary logistic Regression classifier before, the Softmax classifier is its generalization to multiple classes. Unlike the SVM which treats the outputs $f(x_i, W)$ as (uncalibrated and possibly difficult to interpret) scores for each class, the Softmax classifier gives a slightly more intuitive output (normalized class probabilities) and also has a probabilistic interpretation that we will descrive shortly. In the Softmax classifier, the function mapping $F(x_i; W) = Wx_i$ stays unchanged, but we now interpreted these scores as the unormalized log probabilities for each class and replace the hinge loss with a **cross-entropy loss** that has the form 

$$ L_i = -\log\left(\frac{e^{f_{y_i}}}{ \sum_j e^{f_j} }\right) \hspace{0.5in} \text{or equivalently} \hspace{0.5in} L_i = -f_{y_i} + \log\sum_j e^{f_j}$$